# Patient EMPI Graph
This notebook accompanies our blog post. Ingest into an Amazon Neptune database, and then query, patient data based on Open Enterprise Master Patient Index (OpenEMPI) model. See https://www.openhealthnews.com/content/openempi for more.

This notebook uses a converter tool to transform large JSON export files to CSV for bulk load into Neptune. 

Refer to the blog post to delve deeper into the approach.

## Convert Source Data to Neptune CSV
Convert large JSON files, representing Patients, exported from OrientDB. For demonstration purposes, this is mock data. Convert the files to CSV. Stage in S3 bucket. Bulk-load to Neptune.


### Find name of your S3 Bucket
An S3 bucket for staging patient data was created during setup. Get the name of the bucket.

In [ ]:
import os
import subprocess

stream = os.popen("source ~/.bashrc ; echo $S3_SOURCE_BUCKET; echo $S3_SOURCE_BUCKET")
S3_BUCKET = stream.read().split("\n")[0]

S3_BUCKET

### Grab local copy of S3 data
Get the JSON export as a local file on this instance.

In [ ]:
%%bash -s "$S3_BUCKET"

echo $1
mkdir converter_dir
cd converter_dir
aws s3 sync s3://$1 .
rm -r notebook # already have it
cd converter


### Install NPM dependencies for converter
The Converter uses Node.js. Setup NPM dependencies.

In [ ]:
%%bash

cd converter_dir/converter

# need nvm
curl https://raw.githubusercontent.com/creationix/nvm/master/install.sh | bash
export NVM_DIR="$HOME/.nvm"
    [ -s "$NVM_DIR/nvm.sh" ] && \. "$NVM_DIR/nvm.sh"  # This loads nvm
    [ -s "$NVM_DIR/bash_completion" ] && \. "$NVM_DIR/bash_completion"  # This loads nvm bash_completion
    
# need node 16
nvm install 16

# need dependencies
npm install #dependencies

# let's see which node versions we have
ls -l /home/ec2-user/.nvm/versions/node


### Run the converter
Run the converter through bash shell. Input data file was downloaded to this instance above. Output is also a local file. 

In [ ]:
%%bash

cd converter_dir/converter

/home/ec2-user/.nvm/versions/node/*/bin/node process.js ../data/openempi_patient_db.json.gz patients


### Move converted files to s3
Move the output of the converter (CSV files) to S3 bucket.

In [ ]:
%%bash -s "$S3_BUCKET"

cd converter_dir/converter 
aws s3 cp neptune-patients-identifier.csv.gz s3://$1/data/converted/neptune-patients-identifier.csv.gz
aws s3 cp neptune-patients-patient.csv.gz s3://$1/data/converted/neptune-patients-patient.csv.gz
aws s3 cp neptune-patients-recordLink.csv.gz s3://$1/data/converted/neptune-patients-recordLink.csv.gz
aws s3 cp neptune-patients-identifierEdge.csv.gz s3://$1/data/converted/neptune-patients-identifierEdge.csv.gz


## Load data into Neptune
Bulk-load converter output to Neptune database

In [ ]:
%load -s s3://{S3_BUCKET}/data/converted --store-to loadres --run

## Check load status
Check the bulk load succeeded. There are a few records with missing links. That is to be expected. 

In [ ]:
%load_status {loadres['payload']['loadId']} --details --errors

## Query the Data in Neptune
The patient data is now in Neptune. Let's query it using common access patterns.

### Find patient by SSN. Show patient details, identifiers, record links.
Find the patient with given SSN. Also bring in their identifiers and links to other patient records.

In [ ]:
%%gremlin

// find patients that match on ssn
g.V().
  hasLabel('patient').has('ssn', '460000320'). // show full patient record and summary identifiers, record links
  project('patient', 'identifiers', 'links').
    by(elementMap()).
    by(out('identifierEdge').as('iv').
        select('iv').
        by(elementMap('identifierDomainId', 'identifier')).
      fold()).
    by(outE('recordLink').as('re').
      inV().as('rv').
        select('re', 'rv').
        by(elementMap('weight', 'state', 'source')).
        by(id).fold()).
  limit(100)

### Show a listing of patients, their identifiers, and their links. Patient must have an identifier.
Similar to last query, but now we are looking across all patients. The patient must have an identifier. Limit to 100.

In [ ]:
%%gremlin

// start with all patients
g.V().
  hasLabel('patient'). // check has at least one identifier edge
  where(outE('identifierEdge').
    count().is(gt(0))). // summarize the patient, identifiers, and record links that we get back
  project('patient', 'identifiers', 'links').
    by(elementMap('ssn','postalCode','city','state','givenName','familyName')).
    by(outE('identifierEdge').as('ie').
      inV().as('iv').select('iv').
        by(elementMap('identifierDomainId', 'identifier')).
      fold()).
    by(outE('recordLink').as('re').
      inV().as('rv').
        select('re', 'rv').
        by(elementMap('weight', 'state', 'source')).
        by(id).fold()).
  limit(100)

### Show a listing of patients, their identifiers, and their links. Patient must have a record link.
Similar to last query, but the patient must have a record link. Limit to 100.

In [ ]:
%%gremlin

g.V().hasLabel('patient'). // check at least one record link
  where(out('recordLink')).
  project('patient', 'identifiers', 'links').
    by(elementMap('ssn','postalCode','city','state','givenName','familyName')).
    by(out('identifierEdge').as('iv').
        select('iv').
        by(elementMap('identifierDomainId', 'identifier')).
      fold()).
    by(outE('recordLink').as('re').
      inV().as('rv').
        select('re', 'rv').
        by(elementMap('weight', 'state', 'source')).
        by(id).fold()).
  limit(100)

### Show a listing of patients with multiple record links.
Now let's look at patients who have multiple record links.

In [ ]:
%%gremlin

g.V().hasLabel('patient').
  where(outE('recordLink').
    count().is(gt(1))).
  project('patient', 'identifiers', 'links').
    by(elementMap('ssn','postalCode','city','state','givenName','familyName')).
    by(out('identifierEdge').as('iv').
        select('iv').
        by(elementMap('identifierDomainId', 'identifier')).
      fold()).
    by(outE('recordLink').as('re').
      inV().as('rv').
        select('re', 'rv').
        by(elementMap('weight', 'state', 'source')).
        by(id).fold()).
  limit(100)


### Visualize one patient that has multiple record links.
From results of last query, we see that a specific patient has multiple record links. Run the query to see these links. In the results, select the Graph tab to see a visualization of these links.

In [ ]:
%%gremlin -d T.id -de T.label

// start with patient
g.V('patients-25:3201').
  repeat(outE('identifierEdge', 'recordLink').inV().
    simplePath()).emit().times(10).path().
    by(elementMap()).
  limit(100)